In [1]:
import pandas as pd
import segFunctions

# runs the csv_from_excel function:
# creates csv for lexical access experiment lists
segFunctions.csv_from_excel('Lexical_Access_Experimental_Item_Setup.xlsx','All_LexExperiment_Lists','./data/original_data/exp_files/expLists_all.csv')

#creates a csv for pulling out random selections by word
segFunctions.csv_from_excel('LexicalNonwordStats_Wuggy.xlsx','WuggyBuilt','./data/temp_data/nonwords_all.csv')


<_io.TextIOWrapper name='./data/temp_data/nonwords_all.csv' mode='w' encoding='UTF-8'>

In [15]:
df = pd.read_csv('./data/temp_data/nonwords_all.csv')
i = 0
j = 9
nonwords = []
df.tail()

,Word,Match,Nonword,Nonword_Concat,Rand
795,vol-can,fel-can,felcan,felcan,0.166801
796,vol-can,fil-can,filcan,filcan,0.862816
797,vol-can,fol-dan,foldan,foldan,0.458116
798,vol-can,fop-can,fopcan,fopcan,0.249141
799,vol-can,vuz-can,vuzcan,vuzcan,0.032408


In [41]:
dfTemp = df.loc[i:j,:]
#dfTemp.sort_values(by=['Rand'])
df_sorted = dfTemp.sort_values(by='Rand',ascending = True)
#print(len(dfTemp))
#dfTemp.dtypes
df_sorted


,Word,Match,Nonword,Nonword_Concat,Rand
7,se-gu-ro,te-ru-to,teruto,teruto,0.129886
8,se-gu-ro,te-ru-no,teruno,teruno,0.143047
0,se-gu-ro,te-cu-ra,tecura,tecura,0.298652
4,se-gu-ro,te-vu-to,tevuto,tevuto,0.419774
2,se-gu-ro,te-cu-no,tecuno,tecuno,0.577307
6,se-gu-ro,te-ru-ra,terura,terura,0.611347
9,se-gu-ro,te-du-ra,tedura,tedura,0.617744
1,se-gu-ro,te-cu-to,tecuto,tecuto,0.861067
3,se-gu-ro,te-vu-ra,tevura,tevura,0.941961
5,se-gu-ro,te-vu-no,tevuno,tevuno,0.948716


In [43]:
token = df_sorted.iat[0,3]
print(token)

teruto


In [44]:
print(nonwords)

[]


In [45]:
nonwords.append(token)
print(nonwords)

['teruto']


In [56]:
nonwords = []
i = 0
j = 9
while j < 900:
    df_temp = df.loc[i:j,:]
    df_sorted = df_temp.sort_values(by='Rand',ascending = True)
    token = df_sorted.iat[0,3]
    nonwords.append(token)
    i += 10
    j += 10
print(nonwords)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [74]:
nonwords = []
i = 0
j = 9
k = 0
try:
    while j < 800:
        df_temp = df.loc[i:j,:]
        df_sorted = df_temp.sort_values(by='Rand',ascending = True)
        token = df_sorted.iat[k,3]
        print(token,i,j,k)
        if token in nonwords:
            k += 1
        else:
            nonwords.append(token)
            i += 10
            j += 10
            k = 0
except IndexError:
    print("Upper bound reached")
finally:
    print(nonwords)
    

teruto 0 9 0
lodato 10 19 0
socano 20 29 0
ruceba 30 39 0
comoro 40 49 0
valito 50 59 0
merido 60 69 0
todeca 70 79 0
secoco 80 89 0
deceja 90 99 0
tacuro 100 109 0
cemoca 110 119 0
nosala 120 129 0
ramena 130 139 0
sadido 140 149 0
demasa 150 159 0
rarido 160 169 0
tevido 170 179 0
tasivo 180 189 0
fasama 190 199 0
sebada 200 209 0
dodino 210 219 0
cavaco 220 229 0
nemada 230 239 0
soredo 240 249 0
seciro 250 259 0
facido 260 269 0
tamuda 270 279 0
maroña 280 289 0
vunino 290 299 0
cacato 300 309 0
sicezo 310 319 0
nalana 320 329 0
saseta 330 339 0
pamapo 340 349 0
socido 350 359 0
sarapa 360 369 0
toleno 370 379 0
devido 380 389 0
linido 390 399 0
laldon 400 409 0
firtum 410 419 0
vurnar 420 429 0
rorcon 430 439 0
nongal 440 449 0
murdon 450 459 0
rextor 460 469 0
cansin 470 479 0
marnel 480 489 0
vesvor 490 499 0
larton 500 509 0
carsol 510 519 0
culbor 520 529 0
desbez 530 539 0
masday 540 549 0
montal 550 559 0
repcel 560 569 0
banten 570 579 0
bembor 580 589 0
purnar 590 599 0
ja

In [47]:
print(nonwords)

['teruto', 'teruto', 'lodato', 'socano', 'ruceba', 'comoro', 'valito', 'merido', 'todeca', 'secoco', 'deceja', 'tacuro', 'cemoca', 'nosala', 'ramena', 'sadido', 'demasa', 'rarido', 'tevido', 'tasivo', 'fasama', 'sebada', 'dodino', 'cavaco', 'nemada', 'soredo', 'seciro', 'facido', 'tamuda', 'maroña', 'vunino', 'cacato', 'sicezo', 'nalana', 'saseta', 'pamapo', 'socido', 'sarapa', 'toleno', 'devido', 'linido', 'laldon', 'firtum', 'vurnar', 'rorcon', 'nongal', 'murdon', 'rextor', 'cansin', 'marnel', 'vesvor', 'larton', 'carsol', 'culbor', 'desbez', 'masday', 'montal', 'repcel', 'banten', 'bembor', 'purnar', 'jarnen', 'cuntor', 'pestal', 'martor', 'penril', 'laxtor', 'pestal', 'canvon', 'castes', 'gespan', 'bumpon', 'jucmin', 'dexxin', 'naldon', 'mostel', 'pusnon', 'ralbon', 'sixtan', 'soxtil', 'vuzcan']


In [73]:
print(j)

809


In [72]:
print(len(nonwords))

81


# The Good Stuff
The code block below in this Jupyter Notebook takes the excel file containing nonwords created by the Wuggy.app application and generates a list of nonwords ready for the PyschoPy experiment.

## Wuggy.app
Critical items were taken from Carreiras and Perea (2002) experiment 4. All the critical items fell into one of two structures CV.CV.CV or CVC.CVC where 40 were of the first type and 40 were of the second type for a total of 80 critical items.

This list of 80 critical items were entered into Wuggy.app as words and the second column gave Wuggy.app the correction Spanish syllabification of each critical word—generated from within the application. I then used the verify feature to ensure all data was correct. This resulted in one error because "malena" was not found in the Wuggy.app dictionary. Therefore, I gave the correct syllabification of "ma-le-na" where hypens represent syllable boundaries. I then re-verified the data before runnning the analysis. Once I receive no errors, I ran the following analysis to generate new nonwords. 

### Parameters:

**General Settings:**
* Language module: Orthographic Spanish
* Output type: Only pseudowords
* Maximal number of candidates: 10 per word
* Maximal search time per word: 10 seconds
    
**Output Restrictions:**
* Checked: Match length of subsyllabic segements
* Checked: Match letter length
* Checked: Match transition frequencies (concentric search)
* Checked: Match subsyllabic segments: "2" out of "3"
        
**Output Options:**
* *Syllables* chosen from dropdown box
* Checked: Lexicality
* Checked: OLD20
* Checked: Neighbors at edit distance 1
* Checked: Number of overlapping segments
* Checked: Deviation statistics
        
### Output file

Wuggy outputs a file that contains all the raw data that was then imported into Microsoft Excel to preserve all the unicode characters contained in the data. **Do not simply open this from Windows Explorer or Finder** as it will replace all encoding of the file eliminating special characters.

**WARNING:** Wuggy uses syllabification and it removes the accented vowels in Spanish. The only way to remedy this problem is to fix it once it has been imported into Excel. 

Once in Excel, I renamed the worksheet *WuggyRawOutput*. Then I duplicated this worksheet and renamed the new worksheet *WuggyBuilt*. I then deleted all the columns except for columns A and B ("Word" and "Match"). I then added two columns called "Nonword_Concat" and "Rand".
In the *Nonword_Concat* column, I entered the following formulas:

* For CV-CV-CV words: =CONCATENATE(LEFT(B2,2),MID(B2,4,2),MID(B2,7,2))
    * This formula combines the two leftmost characters, skips character 3 (the first syllable boundary marker), adds the 4th and 5th characters to the string store, skips the 6th character (the second syllable boundary marker), and finally adds the 7th and 8th characters from the left to create the six-letter nonword.


* For CVC-CVC words: =CONCATENATE(LEFT(B403,3),RIGHT(B403,3))
    * This formula combines the three leftmost and three rightmost characters to create the six-letter nonword excluding the one and only syllable boundary marker.

These two formulas are necessary in order to create nonwords without the syllable markers. Once these steps have been completed, the file is ready for the python code housed in the cell below.

## Python Code

The python code here first imports the packages and functions needed to excute all the code correctly:

* pandas - an package to manipulate tabular data
* segFunctions - a function that creates csv files extracted from an Excel Workbook.

The script then calls segFunctions to create a temporary csv file that is then read in and stored as *df*. I then create an empty list to store the words I will extract with the script and setup three different counters. *i* is the counter that I will use reference the rows (minimum range value) in my dataframe and since python is zero-based, I set its default value to zero (0). *j* is the counter that I will use to reference the rows (maximum range value). **Note: this only works when the maximum number of candidates of 10 was actually reached when the Wuggy.app analysis ran** *This can be checked by multiplying the number of words submitted to Wuggy.app by 10. k* is the counter that will be used to reference the first row of the sorted dataframes and will only increase when duplicate nonword entries are found. Once the it has resolved the nonword duplicates it is reset to zero(0) for the next iteration.

Once the preamble is squared away and checks have been made to ensure datastructure is accurate, the meat of the code is executed. While *j* is less the 800, *df_temp* is stored as all columns from *df* with rows between *i* and *j*. Then *df_sorted* stores a new dataframe organized in ascending order according to the *Rand* column, which contains random numbers between 0 and 1, generated from Excel. Using the *df_sorted* dataframe, *token* stores the value from row *k*,zero(0) by default, and column 3. If the token is found in the list *nonwords*, it increases the value of *k* by one to go to the next row and tests it again. This process occurs until *token* is not found in the list. When *token* does not exist in the list, it appends the value of *token* to the list *nonwords* and then increases the values of *i and j* by 10 to get to the next set of data. This process repeats through the entire dataframe. In the current state, the length of the list *nonwords* should be 80. This can be checked using the following code snippet:

```len(nonwords)```

Following all the iterations, the list is added to a dataframe called *df_nonwords*, which is then passed through to a csv file stored in the *./processed_data/exp_files/* directory for later use.  


In [79]:
import pandas as pd
import segFunctions

# runs the csv_from_excel function:
# creates a csv for pulling out random selections by word
segFunctions.csv_from_excel('LexicalNonwordStats_Wuggy.xlsx','WuggyBuilt','./data/temp_data/nonwords_all.csv')

# reads in the csv created by csv_from_excel function
df = pd.read_csv('./data/temp_data/nonwords_all.csv')

nonwords = [] #creates an empty list for appending nonwords
i = 0 # counter used in row delimiter of dataframe
j = 9 # counter used in column delimiter of dataframe
k = 0 # counter used to iterate to next item when duplicate found

#iterate through the dataframe of 800 nonwords generated with Wuggy.app
try:
    while j < 800: # stop iteration before index value is exceeded
        df_temp = df.loc[i:j,:] # subset dataframe by rows with all columns
        df_sorted = df_temp.sort_values(by='Rand',ascending = True) # sort ascending by Rand column
        token = df_sorted.iat[k,3] # store the value of cell in dataframe
        if token in nonwords: # check to see if word is already in list
            k += 1 # when duplicate found go to next row value and try again
        else: # no duplicate found run this
            nonwords.append(token) # append the new token to the list 'nonwords'
            i += 10 #increase by 10 to get to next dataset
            j += 10 #increase by 10 to get to next dataset
            k = 0 #reset row indexer to zero(0) 
except IndexError: # check to see that dataframe length has not been exceeded
    print("Upper bound reached") # print debug message
finally: # once everything is completed, print the list
    # passes the list to a dataframe, which is then written to a csv file
    df_nonwords = pd.DataFrame(data={"col1": nonwords})
    df_nonwords.to_csv('./data/processed_data/exp_files/nonwords_list.csv',header=False)

In [14]:
import pandas as pd
import segFunctions

# runs the csv_from_excel function:
# creates a csv for pulling out random selections by word
segFunctions.csv_from_excel('LexicalNonwordStats_Wuggy.xlsx','WuggyBuilt','./data/temp_data/nonwords_all.csv')

# reads in the csv created by csv_from_excel function
df = pd.read_csv('./data/temp_data/nonwords_all.csv')
print(df)
nonwords = [] #creates an empty list for appending nonwords
i = 0 # counter used in row delimiter of dataframe
j = 9 # counter used in column delimiter of dataframe
k = 0 # counter used to iterate to next item when duplicate found

#iterate through the dataframe of 800 nonwords generated with Wuggy.app
try:
    while j < len(df): # stop iteration before index value is exceeded
        df_temp = df.loc[i:j,:] # subset dataframe by rows with all columns
        df_sorted = df_temp.sort_values(by='Rand',ascending = True) # sort ascending by Rand column
        token = df_sorted.iat[k,3] # store the value of cell in dataframe
        if token in nonwords: # check to see if word is already in list
            k += 1 # when duplicate found go to next row value and try again
        else: # no duplicate found run this
            nonwords.append(token) # append the new token to the list 'nonwords'
            i += 10 #increase by 10 to get to next dataset
            j += 10 #increase by 10 to get to next dataset
            k = 0 #reset row indexer to zero(0)
        print(j)
        
except IndexError: # check to see that dataframe length has not been exceeded
    print('Upper bound reached')

         Word     Match Nonword Nonword_Concat      Rand
0    se-gu-ro  te-cu-ra  tecura         tecura  0.298652
1    se-gu-ro  te-cu-to  tecuto         tecuto  0.861067
2    se-gu-ro  te-cu-no  tecuno         tecuno  0.577307
3    se-gu-ro  te-vu-ra  tevura         tevura  0.941961
4    se-gu-ro  te-vu-to  tevuto         tevuto  0.419774
5    se-gu-ro  te-vu-no  tevuno         tevuno  0.948716
6    se-gu-ro  te-ru-ra  terura         terura  0.611347
7    se-gu-ro  te-ru-to  teruto         teruto  0.129886
8    se-gu-ro  te-ru-no  teruno         teruno  0.143047
9    se-gu-ro  te-du-ra  tedura         tedura  0.617744
10   bo-ni-to  to-sa-to  tosato         tosato  0.712559
11   bo-ni-to  to-ma-to  tomato         tomato  0.999413
12   bo-ni-to  vo-ca-to  vocato         vocato  0.856972
13   bo-ni-to  vo-ra-to  vorato         vorato  0.284149
14   bo-ni-to  vo-da-to  vodato         vodato  0.417282
15   bo-ni-to  vo-sa-to  vosato         vosato  0.707577
16   bo-ni-to  vo-ma-to  vomato

In [18]:
set_test = set(df.columns.values[0])
print(set_test)

{'d', 'r', 'W', 'o'}


In [28]:
print(df.columns.values[3])

Nonword_Concat


In [24]:
pd.unique(df.columns.values[0:3])


array(['Word', 'Match', 'Nonword'], dtype=object)

In [25]:
set(df.columns)

{'Match', 'Nonword', 'Nonword_Concat', 'Rand', 'Word'}

In [27]:
set(df.columns.values[3])

{'C', 'N', '_', 'a', 'c', 'd', 'n', 'o', 'r', 't', 'w'}

In [9]:
import pandas as pd
import segFunctions

# runs the csv_from_excel function:
# creates a csv for pulling out random selections by word
segFunctions.csv_from_excel('LexicalNonwordStats_Wuggy.xlsx','WuggyBuilt','./data/temp_data/nonwords_all.csv')

# reads in the csv created by csv_from_excel function
df = pd.read_csv('./data/temp_data/nonwords_all.csv')

uniquewords = [] #creates an empty list for appending nonwords
k = 0 # counter used to iterate to next item when duplicate found

#print(k)
while k < len(df): # stop iteration before index value is exceeded
    token = df.iat[k,0] # store the value of cell in dataframe
    if token in uniquewords: # check to see if word is already in list
        k += 1
        continue
    else: # no duplicate found run this
        uniquewords.append(token) # append the new token to the list 'nonwords'
        #print(k, token)
        k += 1 #reset row indexer to zero(0)
print(uniquewords)
print(len(uniquewords))

['se-gu-ro', 'bo-ni-to', 'mo-li-no', 'pu-re-za', 'co-ro-na', 'ba-ra-to', 'pe-sa-do', 'do-ce-na', 'pe-no-so', 're-ce-lo', 'ba-su-ra', 'ce-lo-so', 'go-ri-la', 'ma-le-ta', 'pa-ra-do', 're-sa-ca', 'sa-la-do', 'se-na-do', 'ta-ma-ño', 'va-si-ja', 'me-di-da', 'ro-ma-no', 'ca-si-no', 'he-ri-da', 'mo-ne-da', 'de-li-to', 'ga-na-do', 'sa-lu-do', 'pa-lo-ma', 'gu-sa-no', 'ca-se-ta', 'di-se-ño', 'ha-ri-na', 'ma-le-na', 'pa-si-vo', 'ro-sa-do', 'sa-li-va', 'so-ne-to', 're-ca-do', 'vi-sa-do', 'bal-con', 'vir-tud', 'vul-gar', 'rin-con', 'nor-mal', 'per-don', 'sec-tor', 'car-min', 'mar-fil', 'fer-vor', 'bas-ton', 'car-tel', 'cur-sor', 'des-liz', 'mal-dad', 'pos-tal', 'rep-til', 'sar-ten', 'tam-bor', 'pul-gar', 'jar-din', 'pin-tor', 'men-tal', 'pas-tor', 'per-fil', 'fac-tor', 'por-tal', 'car-bon', 'cor-tes', 'des-van', 'bom-bon', 'jaz-min', 'del-fin', 'hal-con', 'pas-tel', 'pul-mon', 'sal-mon', 'sul-tan', 'tex-til', 'vol-can']
80
